In [1]:
import sys
sys.path.append("..")

from methods.dcbo import DynCausalBayesOpt
from utils.sequential_sampling import draw_samples_from_sem_dev
from causal_graph.example_dyn_graphs import three_step_stat
from sem.stationary import StationaryModel_dev
from collections import OrderedDict
from utils.costs import equal_cost
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_probability as tfp
from collections import OrderedDict
tfd = tfp.distributions
tf.random.set_seed(0)

In [2]:
sem_model = StationaryModel_dev()
num_samples = 50
temporal_index = 2
full_samples = OrderedDict([(key, []) for key in sem_model.static().keys()])
epsilon = OrderedDict(
    [
        (key, tfd.Normal(0.0, 0.5).sample((num_samples, temporal_index+1)))
        for key in full_samples.keys()
    ]
)

D_obs = draw_samples_from_sem_dev(sem_model, num_samples, temporal_index, epsilon=epsilon)

In [3]:
dyn_graph = three_step_stat()

D_intervene_ini = OrderedDict()
intervention_domain = OrderedDict([("X", [-5.0, 5.0]), ("Z", [-5.0, 20.0])])
num_trials = 20
task = "min"
cost_fn = equal_cost
num_anchor_points = 100
num_monte_carlo = 100
jitter = 1e-6
dcbo = DynCausalBayesOpt(
    dyn_graph,
    sem_model,
    D_obs,
    D_intervene_ini,
    intervention_domain,
    num_trials,
    task,
    cost_fn,
    num_anchor_points,
    num_monte_carlo,
    jitter,
)

In [4]:
opt_history = dcbo.run()

Early stopping at step 17658
Early stopping at step 13475
Early stopping at step 13475
Dynamic causal Bayesian optimization at time step 0 is started.
Temporal index: 0 . Trial: 0
Intervened exploration set: ('X',) . Intervention point: [[-4.9342585]]
Target variable value: 0.73752856
Optimal value: 0.73752856
Early stopping at step 26
Temporal index: 0 . Trial: 1
Intervened exploration set: ('Z',) . Intervention point: [[2.6706886]]
Target variable value: -1.7661558
Optimal value: -1.7661558
Early stopping at step 26
Early stopping at step 24
Temporal index: 0 . Trial: 2
Intervened exploration set: ('Z',) . Intervention point: [[2.0626736]]
Target variable value: -1.3742878
Optimal value: -1.7661558
Early stopping at step 26
Early stopping at step 24
Temporal index: 0 . Trial: 3
Intervened exploration set: ('Z',) . Intervention point: [[2.7077684]]
Target variable value: -1.7807415
Optimal value: -1.7807415
Early stopping at step 26
Early stopping at step 42
Temporal index: 0 . Trial:

SystemError: \Objects\listobject.c:336: bad argument to internal function

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(15, 3))
for i in range(3):
    axs[i].plot(opt_history[i])
plt.show()